# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - CRUD Operations

**Subtopic** - Integrating SQLAlchemy with Flask for Managing Records using CRUD Operations

**Use Case** - Create a paginated product catalog using Flask and SQLAlchemy, supporting CRUD operations on product records with efficient pagination of results.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
SQLAlchemy==2.0.41
```


# Prompt

## **Problem Overview**

You are tasked with implementing a **real product catalog system** using **Python with SQLAlchemy ORM** and **SQLite**. The catalog must support:

* **Full CRUD operations**: Create, Read, Update, Delete
* **Filtering**: by price range, name substring, and minimum stock
* **Sorting**: by `price`, `name`, or `stock`, in ascending or descending order
* **Pagination**: using page number and page size

Each product is stored in a SQLite database using SQLAlchemy and includes the following fields:

* `id` (Primary Key, Integer, Auto-incremented)
* `name` (String, Required, Non-empty)
* `description` (String, Optional, defaults to empty string)
* `price` (Float, Non-negative)
* `stock` (Integer, Zero or more)

You must implement a **`ProductCatalog`** class that connects to the database and exposes methods that simulate REST API behavior through function calls.


## **Input Format**

The following methods must be implemented:

```python
class ProductCatalog:
    def create_product(self, data: dict) -> dict: ...
    def get_product(self, product_id: int) -> dict: ...
    def update_product(self, product_id: int, data: dict) -> dict: ...
    def delete_product(self, product_id: int) -> dict: ...
    def list_products(
        self,
        page: int = 1,
        page_size: int = 10,
        sort_by: str = None,
        ascending: bool = True,
        min_price: float = None,
        max_price: float = None,
        name_contains: str = None,
        min_stock: int = None
    ) -> dict: ...
```


## **Output Format**

Each method returns a dictionary simulating a JSON API response.

Example `list_products()` result:

```python
{
    "products": [
        {"id": 1, "name": "Monitor", "price": 299.99, "stock": 20}
    ],
    "page": 1,
    "page_size": 10,
    "total_pages": 1,
    "total_products": 1
}
```

Example error:

```python
{"error": "Invalid price"}
```


## **Examples**

### Create a product:

```python
create_product({
    "name": "Monitor",
    "price": 299.99,
    "stock": 10,
    "description": "4K UHD Display"
})
```

**Output:**

```python
{
    "id": 1,
    "name": "Monitor",
    "price": 299.99,
    "stock": 10,
    "description": "4K UHD Display"
}
```

# Requirements


## **Explicit Requirements**

* Use **SQLAlchemy ORM** for all database operations.
* Use **SQLite** for local database storage.
* Validate that:

  * `name` is a non-empty string.
  * `price` is a float >= 0.
  * `stock` is an int >= 0.
  * `description` is a string.
* Filtering and sorting must be applied **before pagination**.
* Sorting should support the following fields: `name`, `price`, `stock`.
* Default values:

  * `description` defaults to an empty string.
  * Pagination defaults: `page=1`, `page_size=10`.
* Maximum allowed `page_size` is 100.
* Raise appropriate errors for invalid inputs or missing products.


## **Implicit Requirements**

* Use SQLAlchemy's `declarative_base()` to define models.
* Use session management properly (create and commit sessions).
* Avoid SQL injection by using ORM-safe expressions.
* Truncate decimals where needed using `float()` in JSON output.
* Convert SQLAlchemy models to Python dicts for output.


## **Edge Cases**

* `page_size` less than 1 or greater than 100 -> return error
* Request for non-existent product -> return `{"error": "Product not found"}`
* Sorting by an invalid field -> return error
* Empty result from filter -> return empty product list with metadata
* `min_price > max_price` -> return empty list


## **Function Signatures**

```python
class ProductCatalog:
    def create_product(self, data: dict) -> dict: ...
    def get_product(self, product_id: int) -> dict: ...
    def update_product(self, product_id: int, data: dict) -> dict: ...
    def delete_product(self, product_id: int) -> dict: ...
    def list_products(
        self,
        page: int = 1,
        page_size: int = 10,
        sort_by: str = None,
        ascending: bool = True,
        min_price: float = None,
        max_price: float = None,
        name_contains: str = None,
        min_stock: int = None
    ) -> dict: ...
```


## **Constraints**

* Use **only standard Python 3.10+** and **SQLAlchemy (2.0+)**
* Use **SQLite** as the backing database
* Do not use any Flask/Django or web frameworks
* Maximum products allowed: 100,000
* Field constraints:

  * `name`: 1–100 characters
  * `price`: 0 <= price
  * `stock`: 0 <= stock
* Sorting must only be allowed on: `name`, `price`, `stock`

In [ ]:
# code

from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    String,
    Float,
    asc,
    desc,
)
from sqlalchemy.orm import sessionmaker, Session, declarative_base
from sqlalchemy.exc import IntegrityError

Base = declarative_base()


class Product(Base):
    __tablename__ = "products"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False)
    description = Column(String, default="")
    price = Column(Float, nullable=False)
    stock = Column(Integer, nullable=False)


class ProductCatalog:
    def __init__(self, database_url: str = "sqlite:///product_catalog.db"):
        self.engine = create_engine(database_url, echo=False)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def create_product(self, data: dict) -> dict:
        if not isinstance(data.get("name"), str) or not data["name"].strip():
            return {"error": "Invalid or missing product name"}

        if len(data["name"]) > 100:
            return {"error": "Product name must be at most 100 characters"}

        if not isinstance(data.get("price"), (int, float)) or data["price"] < 0:
            return {"error": "Invalid price"}

        if not isinstance(data.get("stock"), int) or data["stock"] < 0:
            return {"error": "Invalid stock"}

        description = data.get("description", "")
        if not isinstance(description, str):
            return {"error": "Invalid description"}

        product = Product(
            name=data["name"].strip(),
            description=description.strip(),
            price=float(data["price"]),
            stock=int(data["stock"]),
        )

        session: Session = self.Session()
        try:
            session.add(product)
            session.commit()
            return self._product_to_dict(product)
        except IntegrityError:
            session.rollback()
            return {"error": "Integrity error while creating product"}
        finally:
            session.close()

    def get_product(self, product_id: int) -> dict:
        session: Session = self.Session()
        try:
            product = session.get(Product, product_id)
            if product:
                return self._product_to_dict(product)
            return {"error": "Product not found"}
        finally:
            session.close()

    def update_product(self, product_id: int, data: dict) -> dict:
        session: Session = self.Session()
        try:
            product = session.get(Product, product_id)
            if not product:
                return {"error": "Product not found"}

            if "name" in data:
                if not isinstance(data["name"], str) or not data["name"].strip():
                    return {"error": "Invalid product name"}
                if len(data["name"]) > 100:
                    return {"error": "Product name must be at most 100 characters"}
                product.name = data["name"].strip()

            if "price" in data:
                if not isinstance(data["price"], (int, float)) or data["price"] < 0:
                    return {"error": "Invalid price"}
                product.price = float(data["price"])

            if "stock" in data:
                if not isinstance(data["stock"], int) or data["stock"] < 0:
                    return {"error": "Invalid stock"}
                product.stock = int(data["stock"])

            if "description" in data:
                if not isinstance(data["description"], str):
                    return {"error": "Invalid description"}
                product.description = data["description"].strip()

            session.commit()
            return self._product_to_dict(product)
        except IntegrityError:
            session.rollback()
            return {"error": "Integrity error while updating product"}
        finally:
            session.close()

    def delete_product(self, product_id: int) -> dict:
        session: Session = self.Session()
        try:
            product = session.get(Product, product_id)
            if not product:
                return {"error": "Product not found"}

            session.delete(product)
            session.commit()
            return {"message": "Product deleted successfully"}
        except Exception as exc:
            session.rollback()
            return {"error": f"Could not delete product: {str(exc)}"}
        finally:
            session.close()

    def list_products(
        self,
        page: int = 1,
        page_size: int = 10,
        sort_by: str = None,
        ascending: bool = True,
        min_price: float = None,
        max_price: float = None,
        name_contains: str = None,
        min_stock: int = None,
    ) -> dict:
        if page < 1:
            return {"error": "Page number must be >= 1"}
        if page_size < 1 or page_size > 100:
            return {"error": "Page size must be between 1 and 100"}

        session: Session = self.Session()
        try:
            query = session.query(Product)

            # Filters
            if min_price is not None:
                query = query.filter(Product.price >= min_price)
            if max_price is not None:
                query = query.filter(Product.price <= max_price)
            if name_contains:
                query = query.filter(Product.name.ilike(f"%{name_contains}%"))
            if min_stock is not None:
                query = query.filter(Product.stock >= min_stock)

            # Sorting
            valid_sort_fields = {"name", "price", "stock"}
            if sort_by:
                if sort_by not in valid_sort_fields:
                    return {"error": "Invalid sort field"}
                sort_column = getattr(Product, sort_by)
                if ascending:
                    query = query.order_by(asc(sort_column), asc(Product.id))
                else:
                    query = query.order_by(desc(sort_column), asc(Product.id))
            else:
                query = query.order_by(asc(Product.id))

            total_products = query.count()
            total_pages = (total_products + page_size - 1) // page_size

            query = query.offset((page - 1) * page_size).limit(page_size)
            products = query.all()

            return {
                "products": [self._product_to_dict(product) for product in products],
                "page": page,
                "page_size": page_size,
                "total_pages": total_pages,
                "total_products": total_products,
            }
        finally:
            session.close()

    @staticmethod
    def _product_to_dict(product: Product) -> dict:
        return {
            "id": product.id,
            "name": product.name,
            "description": product.description,
            "price": float(product.price),
            "stock": product.stock,
        }

In [ ]:
# tests
"""Test suite for the ProductCatalog class."""

import unittest
from main import ProductCatalog


class TestProductCatalog(unittest.TestCase):
    """Unit tests for the ProductCatalog class."""

    def setUp(self):
        """Create a fresh in-memory catalog before each test."""
        self.catalog = ProductCatalog("sqlite:///:memory:")

    def tearDown(self):
        """Dispose the database engine after each test."""
        self.catalog.engine.dispose()

    def test_create_valid_product(self):
        """Test creating a product with valid data."""
        product = self.catalog.create_product({
            "name": "Widget",
            "description": "A basic widget",
            "price": 9.99,
            "stock": 100
        })
        self.assertIn("id", product)
        self.assertEqual(product["name"], "Widget")

    def test_create_product_missing_name(self):
        """Test creating product with missing name."""
        result = self.catalog.create_product({
            "description": "No name",
            "price": 9.99,
            "stock": 100
        })
        self.assertIn("error", result)

    def test_create_product_blank_name(self):
        """Test creating product with blank name."""
        result = self.catalog.create_product({
            "name": "   ",
            "price": 9.99,
            "stock": 100
        })
        self.assertIn("error", result)

    def test_create_product_name_too_long(self):
        """Test product name exceeding character limit."""
        result = self.catalog.create_product({
            "name": "x" * 101,
            "price": 9.99,
            "stock": 100
        })
        self.assertIn("error", result)

    def test_create_product_invalid_price(self):
        """Test creating product with negative price."""
        result = self.catalog.create_product({
            "name": "BadPrice",
            "price": -5,
            "stock": 100
        })
        self.assertIn("error", result)

    def test_create_product_invalid_stock(self):
        """Test creating product with negative stock."""
        result = self.catalog.create_product({
            "name": "BadStock",
            "price": 9.99,
            "stock": -1
        })
        self.assertIn("error", result)

    def test_get_existing_product(self):
        """Test fetching an existing product."""
        created = self.catalog.create_product({
            "name": "Gadget",
            "price": 5.55,
            "stock": 10
        })
        fetched = self.catalog.get_product(created["id"])
        self.assertEqual(fetched["name"], "Gadget")

    def test_get_nonexistent_product(self):
        """Test fetching a nonexistent product."""
        result = self.catalog.get_product(999)
        self.assertIn("error", result)

    def test_update_product_name_and_price(self):
        """Test updating name and price of a product."""
        created = self.catalog.create_product({
            "name": "OldName",
            "price": 1.99,
            "stock": 10
        })
        updated = self.catalog.update_product(created["id"], {
            "name": "NewName",
            "price": 2.99
        })
        self.assertEqual(updated["name"], "NewName")
        self.assertEqual(updated["price"], 2.99)

    def test_update_product_invalid_price(self):
        """Test updating product with invalid price."""
        created = self.catalog.create_product({
            "name": "Thing",
            "price": 5.0,
            "stock": 5
        })
        result = self.catalog.update_product(created["id"], {"price": -10})
        self.assertIn("error", result)

    def test_update_product_invalid_name(self):
        """Test updating product with blank name."""
        created = self.catalog.create_product({
            "name": "Something",
            "price": 3.5,
            "stock": 5
        })
        result = self.catalog.update_product(created["id"], {"name": ""})
        self.assertIn("error", result)

    def test_update_nonexistent_product(self):
        """Test updating a nonexistent product."""
        result = self.catalog.update_product(999, {"name": "Ghost"})
        self.assertIn("error", result)

    def test_delete_existing_product(self):
        """Test deleting an existing product."""
        created = self.catalog.create_product({
            "name": "ToDelete",
            "price": 1.99,
            "stock": 1
        })
        result = self.catalog.delete_product(created["id"])
        self.assertEqual(result["message"], "Product deleted successfully")

    def test_delete_nonexistent_product(self):
        """Test deleting a nonexistent product."""
        result = self.catalog.delete_product(999)
        self.assertIn("error", result)

    def test_list_products_empty(self):
        """Test listing products when catalog is empty."""
        result = self.catalog.list_products()
        self.assertEqual(result["products"], [])
        self.assertEqual(result["total_products"], 0)

    def test_list_products_pagination(self):
        """Test listing products with pagination."""
        for i in range(25):
            self.catalog.create_product({
                "name": f"Item{i}",
                "price": i,
                "stock": i
            })
        page1 = self.catalog.list_products(page=1, page_size=10)
        page3 = self.catalog.list_products(page=3, page_size=10)
        self.assertEqual(len(page1["products"]), 10)
        self.assertEqual(len(page3["products"]), 5)

    def test_list_products_filter_min_price(self):
        """Test listing products with minimum price filter."""
        self.catalog.create_product({
            "name": "Cheap",
            "price": 1,
            "stock": 5
        })
        self.catalog.create_product({
            "name": "Expensive",
            "price": 100,
            "stock": 5
        })
        result = self.catalog.list_products(min_price=50)
        self.assertEqual(len(result["products"]), 1)
        self.assertEqual(result["products"][0]["name"], "Expensive")

    def test_list_products_filter_name_contains(self):
        """Test listing products by name substring match."""
        self.catalog.create_product({
            "name": "Foobar",
            "price": 10,
            "stock": 5
        })
        self.catalog.create_product({
            "name": "Bazqux",
            "price": 10,
            "stock": 5
        })
        result = self.catalog.list_products(name_contains="bar")
        self.assertEqual(len(result["products"]), 1)
        self.assertEqual(result["products"][0]["name"], "Foobar")

    def test_list_products_sort_by_price_desc(self):
        """Test sorting products by descending price."""
        self.catalog.create_product({"name": "A", "price": 10, "stock": 1})
        self.catalog.create_product({"name": "B", "price": 5, "stock": 1})
        result = self.catalog.list_products(sort_by="price", ascending=False)
        self.assertEqual(result["products"][0]["price"], 10)

    def test_list_products_invalid_sort_field(self):
        """Test listing with an invalid sort field."""
        result = self.catalog.list_products(sort_by="nonexistent")
        self.assertIn("error", result)

    def test_list_products_invalid_page_number(self):
        """Test listing with invalid page number."""
        result = self.catalog.list_products(page=0)
        self.assertIn("error", result)

    def test_list_products_invalid_page_size(self):
        """Test listing with invalid page size."""
        result = self.catalog.list_products(page_size=1000)
        self.assertIn("error", result)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/915052d0-3b85-4781-a0c2-bfdda2c29d67?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67819%2Fview

#### Model code:

```python
from sqlalchemy import create_engine, Column, Integer, String, Float, asc, desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.exc import IntegrityError

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False)
    description = Column(String, default="")
    price = Column(Float, nullable=False)
    stock = Column(Integer, nullable=False)

class ProductCatalog:
    def __init__(self, database_url='sqlite:///product_catalog.db'):
        self.engine = create_engine(database_url, echo=False)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def create_product(self, data: dict) -> dict:
        if not data.get('name') or not isinstance(data['name'], str) or len(data['name']) > 100:
            return {"error": "Invalid or missing name"}

        if 'price' not in data or not isinstance(data['price'], (int, float)) or data['price'] < 0:
            return {"error": "Invalid price"}

        if 'stock' not in data or not isinstance(data['stock'], int) or data['stock'] < 0:
            return {"error": "Invalid stock"}

        description = data.get('description', "")
        product = Product(name=data['name'], description=description, price=data['price'], stock=data['stock']) # <Issue>: No type check for description before creating the product. Violates explicit requirement.
        
        session: Session = self.Session()
        try:
            session.add(product)
            session.commit()
            return self._product_to_dict(product)
        except IntegrityError as e:
            session.rollback()
            return {"error": "Integrity error, could not add product"}
        finally:
            session.close()

    def get_product(self, product_id: int) -> dict:
        session = self.Session()
        product = session.get(Product, product_id)
        session.close()

        if product:
            return self._product_to_dict(product)
        else:
            return {"error": "Product not found"}

    def update_product(self, product_id: int, data: dict) -> dict:
        session = self.Session()
        product = session.get(Product, product_id)

        if not product:
            session.close()
            return {"error": "Product not found"}

        if 'name' in data and (not data['name'] or not isinstance(data['name'], str) or len(data['name']) > 100):
            session.close()
            return {"error": "Invalid name"}

        if 'price' in data and (not isinstance(data['price'], (int, float)) or data['price'] < 0):
            session.close()
            return {"error": "Invalid price"}

        if 'stock' in data and (not isinstance(data['stock'], int) or data['stock'] < 0):
            session.close()
            return {"error": "Invalid stock"}

        product.name = data.get('name', product.name)
        product.description = data.get('description', product.description)
        product.price = data.get('price', product.price)
        product.stock = data.get('stock', product.stock)

        try:
            session.commit()
            return self._product_to_dict(product)
        except IntegrityError as e:
            session.rollback()
            return {"error": "Integrity error, could not update product"}
        finally:
            session.close()

    def delete_product(self, product_id: int) -> dict:
        session = self.Session()
        product = session.get(Product, product_id)

        if not product:
            session.close()
            return {"error": "Product not found"}

        try:
            session.delete(product)
            session.commit()
            return {"message": "Product deleted successfully"}
        except Exception as e:
            session.rollback()
            return {"error": "Could not delete product"}
        finally:
            session.close()

    def list_products(self, page: int = 1, page_size: int = 10, sort_by: str = None, ascending: bool = True,
                      min_price: float = None, max_price: float = None, name_contains: str = None, min_stock: int = None) -> dict:
        if page_size < 1 or page_size > 100:
            return {"error": "Invalid page size"}

        session = self.Session()
        query = session.query(Product)

        # Filtering
        if min_price is not None:
            query = query.filter(Product.price >= min_price)
        if max_price is not None:
            query = query.filter(Product.price <= max_price)
        if name_contains:
            query = query.filter(Product.name.like(f"%{name_contains}%"))
        if min_stock is not None:
            query = query.filter(Product.stock >= min_stock)

        # Sorting
        valid_sort_fields = {'name', 'price', 'stock'}
        if sort_by and sort_by in valid_sort_fields:
            if ascending:
                query = query.order_by(asc(getattr(Product, sort_by)))
            else:
                query = query.order_by(desc(getattr(Product, sort_by)))
        elif sort_by:
            session.close()
            return {"error": "Invalid sort field"}

        # Pagination
        total_products = query.count()
        total_pages = (total_products + page_size - 1) // page_size
        query = query.offset((page - 1) * page_size).limit(page_size)
        
        products = query.all()
        session.close()
        
        return {
            "products": [self._product_to_dict(product) for product in products],
            "page": page,
            "page_size": page_size,
            "total_pages": total_pages,
            "total_products": total_products
        }

    def _product_to_dict(self, product: Product) -> dict:
        return {
            "id": product.id,
            "name": product.name,
            "description": product.description,
            "price": float(product.price),
            "stock": product.stock
        }

```